In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Nov 27 07:08:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext autoreload

In [3]:
!git clone https://github.com/TropComplique/mtcnn-pytorch.git 

Cloning into 'mtcnn-pytorch'...
remote: Enumerating objects: 76, done.
remote: Total 76 (delta 0), reused 0 (delta 0), pack-reused 76
Unpacking objects: 100% (76/76), done.


In [4]:
!git clone https://github.com/TreB1eN/InsightFace_Pytorch.git

Cloning into 'InsightFace_Pytorch'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 14.35 MiB | 29.45 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [35]:
!pip install bcolz
!pip install mxnet

     |████████████████████████████████| 1.5MB 7.1MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2661313 sha256=c2128cbdf5cae7e438433265278094ef404d2b78b492b56552f4da471d2356bb
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz
     |████████████████████████████████| 55.0MB 104kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!sudo tar -xvf '/content/drive/My Drive/Colab Notebooks/project/Equalizedface.tar.gz'

Equalizedface/
Equalizedface/property
Equalizedface/train.idx
Equalizedface/train.rec


In [36]:
%autoreload 2
import glob
import os
import numpy as np
import pandas as pd
import pickle
import cv2
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, roc_auc_score
import random
from pathlib import Path
import argparse
import mxnet as mx
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.nn import functional as fnn
from torch.utils import data
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from InsightFace_Pytorch.config import get_config
from InsightFace_Pytorch.data.data_pipe import load_bin, load_mx_rec


In [9]:
# Получим из .rec папку с изображениями
test_transform = transforms.Compose([
                transforms.ToTensor()
                #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
          ])
data_path = Path('')
rec_path = 'Equalizedface'
rec_path = data_path/rec_path
load_mx_rec(rec_path)

100%|██████████| 1251416/1251416 [22:46<00:00, 915.71it/s]


In [10]:

len(os.listdir('Equalizedface/imgs'))

27999

In [6]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [10]:
! pip install facenet_pytorch

     |████████████████████████████████| 1.9MB 6.8MB/s 


In [11]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter

In [12]:


workers = 0 if os.name == 'nt' else 4

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [23]:
# На обучающем датасете, который был в формате .rec лица уже во всё изображение, поэтому не будем делать дополнительный crop
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization,
    transforms.Resize((160,160))
])
dataset = datasets.ImageFolder('Equalizedface/imgs', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]


In [25]:
len(img_inds), len(dataset.classes)

(1251416, 27999)

In [26]:
batch_size = 64
train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [30]:
len(dataset.class_to_idx)

27999

In [31]:
resnet = InceptionResnetV1(pretrained='vggface2',classify=True)
resnet.logits = torch.nn.Linear(512, len(dataset.class_to_idx))
resnet = resnet.to(device)


In [32]:
# Заморозили всё, кроме последнего слоя, чтобы обучить последний слой
for name, param in resnet.named_parameters():
    if not 'logits' in name:
        param.requires_grad = False

In [33]:
for name, param in resnet.named_parameters():
    print(name, param.requires_grad)

conv2d_1a.conv.weight False
conv2d_1a.bn.weight False
conv2d_1a.bn.bias False
conv2d_2a.conv.weight False
conv2d_2a.bn.weight False
conv2d_2a.bn.bias False
conv2d_2b.conv.weight False
conv2d_2b.bn.weight False
conv2d_2b.bn.bias False
conv2d_3b.conv.weight False
conv2d_3b.bn.weight False
conv2d_3b.bn.bias False
conv2d_4a.conv.weight False
conv2d_4a.bn.weight False
conv2d_4a.bn.bias False
conv2d_4b.conv.weight False
conv2d_4b.bn.weight False
conv2d_4b.bn.bias False
repeat_1.0.branch0.conv.weight False
repeat_1.0.branch0.bn.weight False
repeat_1.0.branch0.bn.bias False
repeat_1.0.branch1.0.conv.weight False
repeat_1.0.branch1.0.bn.weight False
repeat_1.0.branch1.0.bn.bias False
repeat_1.0.branch1.1.conv.weight False
repeat_1.0.branch1.1.bn.weight False
repeat_1.0.branch1.1.bn.bias False
repeat_1.0.branch2.0.conv.weight False
repeat_1.0.branch2.0.bn.weight False
repeat_1.0.branch2.0.bn.bias False
repeat_1.0.branch2.1.conv.weight False
repeat_1.0.branch2.1.bn.weight False
repeat_1.0.branch2

In [34]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}
epochs = 3

In [35]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [36]:
# Обучаю только последний слой
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |  3911/3911 | loss:   10.3968 | fps:  963.6921 | acc:    0.0000   

Epoch 1/3
----------
Train | 15643/15643| loss:    2.1215 | fps:  909.2809 | acc:    0.6537   
Valid |  3911/3911 | loss:    0.9669 | fps:  964.0842 | acc:    0.7991   

Epoch 2/3
----------
Train | 15643/15643| loss:    1.0356 | fps:  914.1127 | acc:    0.7882   
Valid |  3911/3911 | loss:    0.9024 | fps:  963.4849 | acc:    0.8204   

Epoch 3/3
----------
Train | 15643/15643| loss:    0.9524 | fps:  921.4893 | acc:    0.8073   
Valid |  3911/3911 | loss:    0.8906 | fps:  968.7215 | acc:    0.8302   


In [37]:
# Specify a path
PATH = "resnet_equalizedface_freeze.pth"

# Save
torch.save(resnet.state_dict(), PATH)

In [38]:
# Разморозили
for name, param in resnet.named_parameters():
#    if not 'logits' in name:
  param.requires_grad = True

In [39]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5])
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}
epochs = 3

In [40]:
# Разморозили
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |  3911/3911 | loss:    0.8906 | fps:  996.7119 | acc:    0.8302   

Epoch 1/3
----------
Train | 15643/15643| loss:    1.6771 | fps:  320.2097 | acc:    0.6738   
Valid |  3911/3911 | loss:    0.9260 | fps: 1006.8447 | acc:    0.7984   

Epoch 2/3
----------
Train | 15643/15643| loss:    0.8400 | fps:  321.0092 | acc:    0.8102   
Valid |  3911/3911 | loss:    0.6703 | fps:  986.4522 | acc:    0.8499   

Epoch 3/3
----------
Train | 15643/15643| loss:    0.6056 | fps:  320.9830 | acc:    0.8574   
Valid |  3911/3911 | loss:    0.5823 | fps:  975.4481 | acc:    0.8692   


In [43]:
# Specify a path
PATH = "resnet_equalizedface_not_freeze.pth"

# Save
torch.save(resnet.state_dict(), PATH)

In [123]:
# Дообучим ещё ( С этим дообучением получилось качество получше)

optimizer = optim.Adam(resnet.parameters(), lr=0.0003)
scheduler = MultiStepLR(optimizer, [5])
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}
epochs = 3

writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |  3911/3911 | loss:    0.5822 | fps:  980.8099 | acc:    0.8692   

Epoch 1/3
----------
Train | 15643/15643| loss:    0.2042 | fps:  320.7318 | acc:    0.9516   
Valid |  3911/3911 | loss:    0.2857 | fps:  979.4978 | acc:    0.9367   

Epoch 2/3
----------
Train | 15643/15643| loss:    0.1049 | fps:  319.9578 | acc:    0.9739   
Valid |  3911/3911 | loss:    0.2902 | fps:  977.6923 | acc:    0.9367   

Epoch 3/3
----------
Train | 15643/15643| loss:    0.0865 | fps:  320.8113 | acc:    0.9776   
Valid |  3911/3911 | loss:    0.2972 | fps:  982.8597 | acc:    0.9364   


In [125]:
PATH = "/content/drive/My Drive/Colab Notebooks/resnet_equalizedface_all.pth"
torch.save(resnet, PATH)

In [126]:
import copy
old_model = copy.deepcopy(resnet)

##**Загружаем датасет RFW**

In [13]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mila601","key":"d7296722c70eeeb1367e467aad3d6cc7"}'}

In [14]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 63 Nov 27 07:10 kaggle.json


In [15]:
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets list --mine

ref                              title                     size  lastUpdated          downloadCount  
-------------------------------  -----------------------  -----  -------------------  -------------  
mila601/rfw-image                RFW_image                  1GB  2020-10-24 15:41:08             25  
mila601/ijba-made                IJBA_made                  3GB  2020-10-31 12:11:17              1  
mila601/nlp-made-hw5             nlp_made_hw5             844MB  2020-07-03 19:59:45              9  
mila601/made-cv-hw1-test-points  made_cv_hw1_test_points    8MB  2020-05-05 21:02:40             15  


In [17]:
!kaggle datasets download -d mila601/rfw-image

 99% 1.27G/1.28G [00:16<00:00, 94.4MB/s]
100% 1.28G/1.28G [00:16<00:00, 85.4MB/s]


In [18]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: races/RFW_test/data/Indian/m.06znml3/m.06znml3_0003.jpg  
  inflating: races/RFW_test/data/Indian/m.06znml3/m.06znml3_0004.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp02/m.06zp02_0001.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp02/m.06zp02_0002.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp02/m.06zp02_0003.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp02/m.06zp02_0004.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp8lh/m.06zp8lh_0001.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp8lh/m.06zp8lh_0002.jpg  
  inflating: races/RFW_test/data/Indian/m.06zp8lh/m.06zp8lh_0003.jpg  
  inflating: races/RFW_test/data/Indian/m.06zpjqp/m.06zpjqp_0001.jpg  
  inflating: races/RFW_test/data/Indian/m.06zpjqp/m.06zpjqp_0002.jpg  
  inflating: races/RFW_test/data/Indian/m.06zpjqp/m.06zpjqp_0003.jpg  
  inflating: races/RFW_test/data/Indian/m.06zpjqp/m.06zpjqp_0004.jpg  
  inflating: races/RFW_test/data/I

In [19]:
!rm rfw-image.zip

In [98]:
# Взять модель предобученную с гитхаба, чтобы на выходе были эмбединги
#resnet = InceptionResnetV1(pretrained='vggface2',classify=False).eval().to(device)

#**Проверим качество модели на RFW**

In [52]:
# загрузим модель, которую мы дообучили
PATH = "/content/drive/My Drive/Colab Notebooks/resnet_equalizedface_all.pth"
resnet = torch.load(PATH)

In [83]:

class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        #x = F.normalize(x, p=2, dim=1)
        return x
resnet.logits = Identity()
resnet = resnet.eval().to(device)

In [85]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization,
    transforms.Resize((220,220)),
    transforms.CenterCrop((160,160))
])

def preprocess_image(filename, transform):
  img = Image.open(filename)
  imgs = [img]
  imgs = np.stack([np.uint8(img) for img in imgs])
  imgs = torch.as_tensor(imgs.copy())
  processed_tensor = imgs[0]#.permute(0,3,1,2)[0]
  processed_tensor = transform(processed_tensor.cpu())
  return processed_tensor

In [32]:
def read_pairs(path):
  pairs = pd.read_csv(path, header=None)
  pairs = pairs[0]
  rows = []
  for pair in pairs:
    if len(pair.split('\t'))>3:
        row = pair.split('\t')
        rows.append([row[0],row[2],int(row[1]), int(row[3]), 0])
    else:
        row = pair.split('\t')
        rows.append([row[0],row[0], int(row[1]), int(row[2]), 1])    
  data_pair = pd.DataFrame(rows, columns=['person_l','person_r','number_l','number_r','same'])
  return data_pair
  
def make_df_with_paths(path_root):
  dataset = []
  for path in glob.iglob(os.path.join(path_root, "**", "*.jpg")):
      person = path.split("/")[-2]
      dataset.append({"person":person, "path": path})
  print(len(dataset))
  dataset = pd.DataFrame(dataset)
  dataset['number'] = dataset.path.apply(lambda x: int(x.split('_')[-1].split('.')[0])) 
  return dataset
def metrics(result):
  print(f'roc_auc_score = {roc_auc_score(result.target, result.score)}')
  res = {}
  for trashold in np.linspace(0.3,0.9,200):
    result['predict'] = result['score'].apply(lambda x: 1 if x>trashold else 0)
    res[trashold]= accuracy_score(result.target, result.predict)
  print(f'(trashold, accuracy) =  {[(key, value) for (key, value) in res.items() if value==max(res.values())][0]}')

# RFW Caucasian

In [63]:
# Считаем уже отобранные 6000 пар по cosin similarity
caucasian_pais = 'races/RFW_test/txts/Caucasian/Caucasian_pairs.txt'
data_pair = read_pairs(caucasian_pais)
data_pair.head()

,person_l,person_r,number_l,number_r,same
0,m.027n6t8,m.027n6t8,3,2,1
1,m.016lb3,m.016lb3,4,2,1
2,m.02_4qt,m.02_4qt,3,2,1
3,m.09g8fj,m.09g8fj,3,1,1
4,m.01q3xr,m.01q3xr,1,3,1


In [64]:
# Загрузим пути для всех изображений
path_caucasian = 'races/RFW_test/data/Caucasian/'
dataset = make_df_with_paths(path_caucasian)
dataset.head()

10196


,person,path,number
0,m.01r0fmm,races/RFW_test/data/Caucasian/m.01r0fmm/m.01r0...,3
1,m.01r0fmm,races/RFW_test/data/Caucasian/m.01r0fmm/m.01r0...,1
2,m.01r0fmm,races/RFW_test/data/Caucasian/m.01r0fmm/m.01r0...,4
3,m.01r0fmm,races/RFW_test/data/Caucasian/m.01r0fmm/m.01r0...,2
4,m.0n_h8,races/RFW_test/data/Caucasian/m.0n_h8/m.0n_h8_...,1


In [65]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [66]:
data_pair.head()

,person_l,person_r,number_l,number_r,same,path_l,path_r
0,m.027n6t8,m.027n6t8,3,2,1,races/RFW_test/data/Caucasian/m.027n6t8/m.027n...,races/RFW_test/data/Caucasian/m.027n6t8/m.027n...
1,m.016lb3,m.016lb3,4,2,1,races/RFW_test/data/Caucasian/m.016lb3/m.016lb...,races/RFW_test/data/Caucasian/m.016lb3/m.016lb...
2,m.02_4qt,m.02_4qt,3,2,1,races/RFW_test/data/Caucasian/m.02_4qt/m.02_4q...,races/RFW_test/data/Caucasian/m.02_4qt/m.02_4q...
3,m.02_4qt,m.060x66,3,2,0,races/RFW_test/data/Caucasian/m.02_4qt/m.02_4q...,races/RFW_test/data/Caucasian/m.060x66/m.060x6...
4,m.09g8fj,m.09g8fj,3,1,1,races/RFW_test/data/Caucasian/m.09g8fj/m.09g8f...,races/RFW_test/data/Caucasian/m.09g8fj/m.09g8f...


In [67]:
# Собираем в датафрейм эмбединги по всем картинка
features = []
for idx in tqdm(range(dataset.shape[0])):
  filename = dataset.path[idx] 
  processed_image = preprocess_image(filename, trans).unsqueeze(0).to(device)
  preds = resnet(processed_image).detach().cpu()
  features.append([dataset.path[idx]] + list(preds[0]))

columns = ['path'] + ['feature_'+str(i+1) for i in range(512)]
df_features = pd.DataFrame(features, columns = columns)
columns = columns[1:]

100%|██████████| 10196/10196 [11:03<00:00, 15.38it/s]


In [68]:
%time
# Посчитаем косинусное растояние между парами изображений
sim = []
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):
  path_l = data_pair.loc[idx, 'path_l']
  path_r = data_pair.loc[idx, 'path_r'] 
  data_pair.loc[idx, 'sim'] = cosine_similarity(np.array(df_features.loc[df_features.path == path_l,columns]).reshape(1,-1), np.array(df_features.loc[df_features.path == path_r,columns]).reshape(1,-1))[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

  0%|          | 17/6000 [00:00<00:35, 167.47it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


100%|██████████| 6000/6000 [00:37<00:00, 159.26it/s]


На самом деле здесь accuracy не совсем честно считается, так как нужно заранее определить trashold, но я хотела увидеть, какое максимальное accuracy можно получить

In [37]:
# Если использовать предобученную модель с github и не дообучать её
metrics(result)

roc_auc_score = 0.9607660000000001
(trashold, accuracy) =  (0.4477386934673367, 0.895)


In [69]:
# Если использовать дообученную модель на Equalizedface
metrics(result)

roc_auc_score = 0.9459227777777779
(trashold, accuracy) =  (0.43567839195979896, 0.8706666666666667)


# RFW African

In [86]:
# Считаем уже отобранные 6000 пар по cosin similarity
African_pais = 'races/RFW_test/txts/African/African_pairs.txt'
data_pair = read_pairs(African_pais)

# Загрузим пути для всех изображений
path_african = 'races/RFW_test/data/African/'
dataset = make_df_with_paths(path_african)

10415


In [58]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [59]:
# Собираем в датафрейм эмбединги по всем изображениям
features = []
for idx in tqdm(range(dataset.shape[0])):
  filename = dataset.path[idx] 
  processed_image = preprocess_image(filename, trans).unsqueeze(0).to(device)
  preds = resnet(processed_image).detach().cpu()
  features.append([dataset.path[idx]] + list(preds[0]))

columns = ['path'] + ['feature_'+str(i+1) for i in range(512)]
df_features = pd.DataFrame(features, columns = columns)
columns = columns[1:]

100%|██████████| 10415/10415 [11:23<00:00, 15.24it/s]


In [60]:
%time
# Посчитаем косинусное растояние между парами изображений
sim = []
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):
  path_l = data_pair.loc[idx, 'path_l']
  path_r = data_pair.loc[idx, 'path_r'] 
  data_pair.loc[idx, 'sim'] = cosine_similarity(np.array(df_features.loc[df_features.path == path_l,columns]).reshape(1,-1), np.array(df_features.loc[df_features.path == path_r,columns]).reshape(1,-1))[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

  0%|          | 16/6000 [00:00<00:38, 153.89it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


100%|██████████| 6000/6000 [00:38<00:00, 157.52it/s]


In [48]:
# Если использовать предобученную модель с github и не дообучать её 
metrics(result)

roc_auc_score = 0.9026611111111111
(trashold, accuracy) =  (0.5653266331658291, 0.8246666666666667)


In [61]:
# Если использовать дообученную модель на Equalizedface
metrics(result)

roc_auc_score = 0.8935357777777777
(trashold, accuracy) =  (0.5020100502512563, 0.8086666666666666)


# RFW Indian

In [99]:
# Считаем уже отобранные 6000 пар по cosin similarity
indian_pais = 'races/RFW_test/txts/Indian/Indian_pairs.txt'
data_pair = read_pairs(indian_pais)

# Загрузим пути для всех изображений
path_indian = 'races/RFW_test/data/Indian/'
dataset = make_df_with_paths(path_indian)

10308


In [100]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [101]:
# Собираем в датафрейм эмбединги по всем изображениям
features = []
for idx in tqdm(range(dataset.shape[0])):
  filename = dataset.path[idx] 
  processed_image = preprocess_image(filename, trans).unsqueeze(0).to(device)
  preds = resnet(processed_image).detach().cpu()
  features.append([dataset.path[idx]] + list(preds[0]))

columns = ['path'] + ['feature_'+str(i+1) for i in range(512)]
df_features = pd.DataFrame(features, columns = columns)
columns = columns[1:]

100%|██████████| 10308/10308 [11:12<00:00, 15.32it/s]


In [102]:
%time
# Посчитаем косинусное растояние между парами изображений
sim = []
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):
  path_l = data_pair.loc[idx, 'path_l']
  path_r = data_pair.loc[idx, 'path_r'] 
  data_pair.loc[idx, 'sim'] = cosine_similarity(np.array(df_features.loc[df_features.path == path_l,columns]).reshape(1,-1), np.array(df_features.loc[df_features.path == path_r,columns]).reshape(1,-1))[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

  0%|          | 17/6000 [00:00<00:36, 162.33it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


100%|██████████| 6000/6000 [00:37<00:00, 160.16it/s]


In [103]:
# Если использовать предобученную модель с github и не дообучать её 
metrics(result)

roc_auc_score = 0.9096670000000001
(trashold, accuracy) =  (0.5592964824120603, 0.8351666666666666)


In [74]:
# Если использовать дообученную модель на Equalizedface
metrics(result)

roc_auc_score = 0.9203381111111112
(trashold, accuracy) =  (0.48693467336683416, 0.8463333333333334)


# RFW Asian

In [104]:
# Считаем уже отобранные 6000 пар по cosin similarity
asian_pais = 'races/RFW_test/txts/Asian/Asian_pairs.txt'
data_pair = read_pairs(asian_pais)

# Загрузим пути для всех изображений
path_asian = 'races/RFW_test/data/Asian/'
dataset = make_df_with_paths(path_asian)

9688


In [105]:
data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_l', 'number_l'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_l'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

data_pair = data_pair.merge(dataset[['person','number','path']], left_on=['person_r', 'number_r'], right_on=['person', 'number'], how='inner')
data_pair = data_pair.rename({'path': 'path_r'}, axis='columns')
data_pair.drop(['person','number'], axis=1, inplace=True)

In [106]:
# Собираем в датафрейм эмбединги по всем изображениям
features = []
for idx in tqdm(range(dataset.shape[0])):
  filename = dataset.path[idx] 
  processed_image = preprocess_image(filename, trans).unsqueeze(0).to(device)
  preds = resnet(processed_image).detach().cpu()
  features.append([dataset.path[idx]] + list(preds[0]))

columns = ['path'] + ['feature_'+str(i+1) for i in range(512)]
df_features = pd.DataFrame(features, columns = columns)
columns = columns[1:]

100%|██████████| 9688/9688 [10:35<00:00, 15.25it/s]


In [107]:
%time
# Посчитаем косинусное растояние между парами изображений
sim = []
data_pair['sim'] = [0.0]*data_pair.shape[0]
for idx in tqdm(range(data_pair.shape[0])):
  path_l = data_pair.loc[idx, 'path_l']
  path_r = data_pair.loc[idx, 'path_r'] 
  data_pair.loc[idx, 'sim'] = cosine_similarity(np.array(df_features.loc[df_features.path == path_l,columns]).reshape(1,-1), np.array(df_features.loc[df_features.path == path_r,columns]).reshape(1,-1))[0][0]
result = pd.DataFrame({'target':data_pair.same, 'score': data_pair.sim})

  0%|          | 16/6000 [00:00<00:37, 158.63it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


100%|██████████| 6000/6000 [00:37<00:00, 161.02it/s]


In [108]:
# Если использовать предобученную модель с github и не дообучать её 
metrics(result)

roc_auc_score = 0.8881322222222222
(trashold, accuracy) =  (0.6135678391959799, 0.8053333333333333)


In [79]:
# Если использовать дообученную модель на Equalizedface
metrics(result)

roc_auc_score = 0.912874
(trashold, accuracy) =  (0.4537688442211055, 0.8321666666666667)
